In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bge-reranker-finetune-3/config.json
/kaggle/input/bge-reranker-finetune-3/zero_to_fp32.py
/kaggle/input/bge-reranker-finetune-3/trainer_state.json
/kaggle/input/bge-reranker-finetune-3/training_args.bin
/kaggle/input/bge-reranker-finetune-3/tokenizer.json
/kaggle/input/bge-reranker-finetune-3/tokenizer_config.json
/kaggle/input/bge-reranker-finetune-3/model.safetensors
/kaggle/input/bge-reranker-finetune-3/special_tokens_map.json
/kaggle/input/bge-reranker-finetune-3/latest
/kaggle/input/bge-reranker-finetune-3/rng_state.pth
/kaggle/input/bge-reranker-finetune-3/sentencepiece.bpe.model
/kaggle/input/output-with-teacherscore-minedhn/output_with_teacherscore_minedHN.jsonl


In [2]:
import json
full_train_data = []
prompt = "Represent this sentence for searching relevant passages: "

with open("/kaggle/input/output-with-teacherscore-minedhn/output_with_teacherscore_minedHN.jsonl", "r", encoding = "utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            temp_dict = json.loads(line)
            temp_dict['prompt'] = prompt
            full_train_data.append(temp_dict)



In [3]:


# Save to a regular JSON file (list of dicts)
with open("train_data.json", "w", encoding="utf-8") as f:
    json.dump(full_train_data, f, ensure_ascii=False, indent=2)

In [4]:
!pip install -q -U FlagEmbedding[finetune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 816.3 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 34.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00:00:0100:

In [5]:
!pip uninstall -y -q flash-attn

In [6]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")

def truncate_passages(passages, max_length=512):
    """Truncate each passage in the list to max_length tokens."""
    truncated = []
    for passage in passages:
        tokens = tokenizer(
            passage,
            max_length=max_length,
            truncation=True,
            padding=False,
            return_tensors=None  # We only want token ids
        )
        truncated_passage = tokenizer.decode(tokens["input_ids"], skip_special_tokens=True)
        truncated.append(truncated_passage)
    return truncated

# Truncate all pos/neg passages in full_train_data
for item in full_train_data:
    item["pos"] = truncate_passages(item.get("pos", []))
    item["neg"] = truncate_passages(item.get("neg", []))


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [7]:
import json

with open("/kaggle/working/full_train_data.json", "w", encoding="utf-8") as f:
    json.dump(full_train_data, f, ensure_ascii=False, indent=2)

print("✅ Saved to /kaggle/working/full_train_data.json")


✅ Saved to /kaggle/working/full_train_data.json


In [8]:
ds_stage0 = {
    "zero_optimization": {
      "stage": 1
    },
    "fp16": {
      "enabled": False
    },
    "bf16": {
      "enabled": False
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupDecayLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto",
            "total_num_steps": "auto"
        }
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 100,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False,
}

with open("ds_stage0.json", "w") as f:
    json.dump(ds_stage0, f)


In [ ]:
import torch
torch.cuda.empty_cache()
import wandb
wandb.login(key="yourapikey")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: trinhhonhatminh (trinhhonhatminh-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [14]:
!torchrun --nproc_per_node=1 \
  -m FlagEmbedding.finetune.reranker.encoder_only.base \
  --model_name_or_path /kaggle/working/test_encoder_only_base_bge-reranker-v2-m3/checkpoint-2190 \
  --cache_dir $HOME/.cache/huggingface/hub \
  --train_data /kaggle/working/full_train_data.json \
  --cache_path ~/.cache \
  --train_group_size 4 \
  --query_max_len 128 \
  --passage_max_len 512 \
  --pad_to_multiple_of 8 \
  --knowledge_distillation False \
  --output_dir ./test_encoder_only_base_bge-reranker-v2-m3 \
  --overwrite_output_dir \
  --learning_rate 6e-5 \
  --num_train_epochs 1 \
  --per_device_train_batch_size 2 \
  --gradient_accumulation_steps 1 \
  --dataloader_drop_last True \
  --warmup_ratio 0.1 \
  --gradient_checkpointing \
  --weight_decay 0.01 \
  --deepspeed /kaggle/working/ds_stage0.json \
  --logging_steps 1 \
  --save_steps 10000

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-08-11 14:38:44.177708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754923124.202067     542 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754923124.209280     542 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[2025-08-11 14:38:50,959] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-08-11 14:38:52,740] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
[2025-08-11 14:38:52,754] [INFO] [comm.py:821:init_distributed] cdb=None
[2025-08-11 14:38:52,755] [INFO] [comm.py:852:init_distributed] Initializing TorchBackend in DeepSpeed with back